In [1]:
# [Cell 1: .env 로드 및 Agent 임포트]
# .env 파일 로드 (core.py가 로드하지만, 노트북 환경을 위해 한 번 더)
import sys
import os
from dotenv import load_dotenv

# 1. 현재 프로젝트 루트 디렉토리(jesafe/)의 절대 경로를 찾습니다.
# os.getcwd() (현재 폴더)의 부모 폴더('..')가 프로젝트 루트라고 지정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 2. 이 경로를 파이썬 모듈 검색 경로(sys.path)에 추가합니다.
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"프로젝트 루트 경로 추가: {project_root}")

load_dotenv()
print(".env 로드 완료:", os.getenv("DATABASE_URL") is not None)

# (주의: async 함수가 아닌 'invoke' (동기) 함수를 사용해야 함)
from src.services import main_agent_executor

print("Agent 로드 완료.")

프로젝트 루트 경로 추가: /Users/lwj0831/jesafe
.env 로드 완료: True
Agent 로드 완료.


In [2]:
# [Cell 2: 메인 Agent 동기 테스트]
# services.py에서 메인 Agent(main_agent_executor)를 가져옵니다.
from src.services import main_agent_executor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from src.data_loader import _load_jeju_coords

# 테스트용 대화 기록 (API 서버의 chat_history_store 역할)
chat_history = ChatMessageHistory()
_load_jeju_coords()

# --- 1. 날씨 질문 테스트 ---
query1 = "오늘 제주도 날씨 어때?"
response1 = main_agent_executor.invoke({
    "input": query1,
    "chat_history": chat_history.messages
})
print(f"User: {query1}\nAI: {response1['output']}\n")
# 기록 업데이트
chat_history.add_user_message(query1)
chat_history.add_ai_message(response1['output'])


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


✅ [data_loader] 제주도 (KMA 격자) DB 로드 완료 (경로: /Users/lwj0831/jesafe/data/kma_jeju_grid_info.csv)

Invoking: `get_current_weather` with `제주시`


[Tool] '오늘 날씨 (OWM)' 도구 호출됨: 제주시
'제주시'의 오늘 날씨 요약: Currently, the weather is characterized by a temperature of 20°C with a slight breeze, as the wind speed is at 9 meters per second. The humidity is quite high at 92%, and there is light rain with a 1-hour precipitation of 0.97mm. The clouds are covering the sky at 100%, and the visibility is at 4164 meters. The atmospheric pressure is at 1016 hPa, and the UV index is at 0. Overall, it's a relatively cool and wet day with overcast skies and moderate winds. So, if you're heading out, be sure to grab your umbrella and dress warmly to stay comfortable in these conditions.오늘 제주시의 날씨는 다음과 같습니다:
- 온도: 20°C
- 바람: 초속 9미터의 약간의 바람
- 습도: 92%
- 강수량: 최근 1시간 동안 0.97mm의 비
- 구름: 하늘을 100% 덮고 있음
- 가시거리: 4164미터
- 기압: 1016 hPa
- 자외선 지수: 0

전반적으로 쌀쌀하고 비가 오는 날씨이며, 하늘이 흐리고 바람이 적당히 불고 있습니다. 외출 시 우산을 챙기고 따뜻하게 입으시길 권장합니다.

> 

In [3]:
# --- 2. 사고 통계 질문 테스트 ---
query2 = "제주시에서 낙상 사고가 몇 건이야?"
response2 = main_agent_executor.invoke({
    "input": query2,
    "chat_history": chat_history.messages
})
print(f"User: {query2}\nAI: {response2['output']}\n")
chat_history.add_user_message(query2)
chat_history.add_ai_message(response2['output'])


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `jeju_safety_statistics_db` with `제주시에서 낙상 사고 건수`


[Tool] SQL Agent 도구 호출됨: 제주시에서 낙상 사고 건수
Action: sql_db_list_tables
Action Input: jeju_accidentsThe table "jeju_accidents" sounds relevant to the question about accident counts in Jeju City. I need to check the schema of this table to find the appropriate columns for querying accident counts specifically related to falls ("낙상").
Action: sql_db_schema
Action Input: jeju_accidents
CREATE TABLE jeju_accidents (
	"SEASN_NM" TEXT, 
	"TRFC_ACDNT_SE_NM" TEXT, 
	"ETC_TRFC_ACDNT_NM" TEXT, 
	"ACDNT_OCRN_LOT" DOUBLE PRECISION, 
	"ACDNT_OCRN_LAT" DOUBLE PRECISION, 
	"DCLR_MM" BIGINT, 
	"SRIL_ONCR_NM" TEXT, 
	"PTN_OCRN_TYPE_NM" TEXT, 
	"PTN_SYM_SE_NM" TEXT, 
	"ACDNT_OCRN_PLC_NM" TEXT, 
	"GRNDS_SGG_NM" TEXT, 
	"ACDNT_INJR_NM" TEXT, 
	"DCLR_YR" BIGINT
)

/*
3 rows from jeju_accidents table:
SEASN_NM	TRFC_ACDNT_SE_NM	ETC_TRFC_ACDNT_NM	ACDNT_OCRN_LOT	ACDNT_OCRN_LAT	DCLR_MM	SRIL_ONCR_NM	PTN_OCRN_TYPE_NM	PTN_SYM_SE_NM	ACDNT_OCRN_PLC_NM	GRNDS_S

In [4]:
# --- 3. (중요) 대화 맥락(Context) 질문 테스트 ---
query3 = "그럼 그 지역(제주시) 한림읍 날씨는 어때?"
response3 = main_agent_executor.invoke({
    "input": query3,
    "chat_history": chat_history.messages
})
print(f"User: {query3}\nAI: {response3['output']}\n")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `get_current_weather` with `한림읍`


[Tool] '오늘 날씨 (OWM)' 도구 호출됨: 한림읍
'한림읍'의 오늘 날씨 요약: Currently, the weather is characterized by a temperature of 20°C with a slight breeze, as the wind speed is at 9 meters per second. The humidity is quite high at 92%, and there is light rain with a 1-hour precipitation of 0.97mm. The clouds are covering the sky at 100%, and the visibility is at 4164 meters. The atmospheric pressure is at 1016 hPa, and the UV index is at 0. Overall, it's a relatively cool and wet day with overcast skies and moderate winds. So, if you're heading out, be sure to grab your umbrella and dress warmly to stay comfortable in these conditions.한림읍의 오늘 날씨는 다음과 같습니다:
- 온도: 20°C
- 바람: 초속 9미터의 약간의 바람
- 습도: 92%
- 강수량: 최근 1시간 동안 0.97mm의 비
- 구름: 하늘을 100% 덮고 있음
- 가시거리: 4164미터
- 기압: 1016 hPa
- 자외선 지수: 0

전반적으로 쌀쌀하고 비가 오는 날씨이며, 하늘이 흐리고 바람이 적당히 불고 있습니다. 외출 시 우산을 챙기고 따뜻하게 입으시길 권장합니다.

> Finished chain.
User: 그럼 그 지역(제주시) 한림읍 날씨는 어때?
AI: 한림읍의 오늘 날씨는 다음과 같습니다:
- 온도: 20°C
- 바람: 초속 9